### **_libraries_**

In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

#### **_Loading Data_**

In [16]:
def get_historical_data(ticker_symbols, period="max"):
    
    historical_data = {}

    for symbol in ticker_symbols:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        data = data.drop(columns=["Dividends", "Stock Splits"])
        historical_data[symbol] = data

    return historical_data

In [144]:
data = get_historical_data(["NVDA", "JOBY", "SOFI", "PLUG", "AMZN", "COIN", "GOOGL", "MTTR", "CHPT", "BAC", "U", "BABA", "RIO", "SYM", "CRWD", "EXPE", "DAR", "ESTE", "CVS", "LXRX",
                            "PHIN", "BWA", "LTHM", "CB", "WBI", "HARP", "SPLK", "EVR", "IEP", "NWBO", "GXO", "UCTT", "CCJ", "UEC", "PLTR", "NLTX", "ALB", "MP"])

In [145]:
nvda = data["NVDA"]
joby = data["JOBY"]
sofi = data["SOFI"]
plug = data["PLUG"]
amzn = data["AMZN"]
coin = data["COIN"]
goog = data["GOOGL"]
mttr = data["MTTR"]
chpt = data["CHPT"]
bacc = data["BAC"]
unit = data["U"]
baba = data["BABA"]
rioo = data["RIO"]
symm = data["SYM"]
crwd = data["CRWD"]
expe = data["EXPE"]
darr = data["DAR"]
este = data["ESTE"]
cvss = data["CVS"]
lxrx = data["LXRX"]
phin = data["PHIN"]
bwaa = data["BWA"]
lthm = data["LTHM"]
cbbb = data["CB"]
wbii = data["WBI"]
harp = data["HARP"]
splk = data["SPLK"]
evrr = data["EVR"]
iepp = data["IEP"]
nwbo = data["NWBO"]
gxoo = data["GXO"]
uctt = data["UCTT"]
ccjj = data["CCJ"]
uecc = data["UEC"]
pltr = data["PLTR"]
nltx = data["NLTX"]
albb = data["ALB"]
mppp = data["MP"]

### **_Indicator_**

In [19]:
def get_pivot_points(data, preferred_order_left = 1, preferred_order_right = 1):
    
    def identify_pivot_points(data, preferred_order_left=1, preferred_order_right=1):
        if preferred_order_left < 1 or preferred_order_right < 1:
            raise ValueError("Preferred pivot order should be 1 or greater.")

        if "High" not in data.columns or "Low" not in data.columns:
            raise ValueError("Input DataFrame must contain 'High' and 'Low' columns.")

        pivot_column_name = f"Pivot ({preferred_order_left}, {preferred_order_right}) Order"
        data[pivot_column_name] = np.nan

        def is_pivot(high_val, low_val, order_left, order_right):
            left_highs = data["High"].iloc[i - order_left:i]
            right_highs = data["High"].iloc[i + 1:i + order_right + 1]
            left_lows = data["Low"].iloc[i - order_left:i]
            right_lows = data["Low"].iloc[i + 1:i + order_right + 1]

            if high_val > left_highs.max() and high_val > right_highs.max():
                return "High"
            elif low_val < left_lows.min() and low_val < right_lows.min():
                return "Low"
            else:
                return np.nan

        for i in range(preferred_order_left, len(data) - preferred_order_right):
            high_val = data["High"].iloc[i]
            low_val = data["Low"].iloc[i]
            pivot_type = is_pivot(high_val, low_val, preferred_order_left, preferred_order_right)
            if pivot_type:
                data.iloc[i, data.columns.get_loc(pivot_column_name)] = pivot_type

        data = data.dropna(how='all')

        return data
    
    def get_pivot_high_dates(data, preferred_order=1):
        pivot_column_name = f"Pivot {preferred_order} Order"
    
        # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
        pivot_high_df = data.loc[data[pivot_column_name] == "High"]
    
        # Extract the dates from the index and store them in a list
        pivot_high_dates = pivot_high_df.index.tolist()
    
        return pivot_high_dates
    def get_pivot_low_dates(data, preferred_order=1):
        pivot_column_name = f"Pivot {preferred_order} Order"
    
        # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
        pivot_low_df = data.loc[data[pivot_column_name] == "Low"]
    
        # Extract the dates from the index and store them in a list
        pivot_low_dates = pivot_low_df.index.tolist()
    
        return pivot_low_dates
    def get_pivot_high_prices(data, preferred_order=1):
        pivot_column_name = f"Pivot {preferred_order} Order"
    
        # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
        pivot_high_df = data.loc[data[pivot_column_name] == "High"]
    
        # Extract the "High" values and store them in a list
        pivot_high_prices = pivot_high_df["High"].tolist()
    
        return pivot_high_prices
    def get_pivot_low_prices(data, preferred_order=1):
        pivot_column_name = f"Pivot {preferred_order} Order"
    
        # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
        pivot_low_df = data.loc[data[pivot_column_name] == "Low"]
    
        # Extract the "High" values and store them in a list
        pivot_low_prices = pivot_low_df["Low"].tolist()
    
        return pivot_low_prices

    pivots = identify_pivot_points(data, preferred_order_left, preferred_order_right)

    order_tuple = tuple([preferred_order_left, preferred_order_right])

    P_h_dates = get_pivot_high_dates(pivots, order_tuple)
    P_h_prices = get_pivot_high_prices(pivots, order_tuple)
    P_l_dates = get_pivot_low_dates(pivots, order_tuple)
    P_l_prices = get_pivot_low_prices(pivots, order_tuple)

    return pd.DataFrame(dict({"Pivot High Dates":P_h_dates, "Pivot High Prices":P_h_prices})), pd.DataFrame({"Pivot Low Dates":P_l_dates, "Pivot Low Prices":P_l_prices})

In [20]:
def get_sr(data, price_column, date_column, min_price_dif, max_date_dif_days):

    #Empty List To Be Filled With Pairs That Meet The Required Difference.
    pairs = []

    #Converting The Price & Date Columns To Workable Lists.
    prices = data[price_column].tolist()
    dates = data[date_column].tolist()

    #Looping Through A Loop To Compare All Price Differences.
    for i in range(len(prices)):
        for j in range(i + 1, len(prices)):
            if abs(prices[i] - prices[j]) < min_price_dif:
                pairs.append((dates[i], prices[i], dates[j], prices[j]))

    #Creating A DataFrame With Pairs Of Selected Pivot Prices And Their Dates.
    selected = pd.DataFrame(pairs, columns=[date_column + "_1", price_column + "_1", date_column + "_2", price_column + "_2"])

    #Convert max_date_dif_days To Timedelta
    max_date_dif = pd.Timedelta(days=max_date_dif_days)

    #Empty List To Be Filled With Rows That Don't Meet A Second Set Of Requirements
    rows_to_remove = []

    #Looping Through The Dates Of The Selected Pivots And Comparing Their Absolute Difference With A Parameter.
    for i in range(len(selected)):
        date_1 = pd.to_datetime(selected[date_column + "_1"].iloc[i])
        date_2 = pd.to_datetime(selected[date_column + "_2"].iloc[i])
        
        date_diff = abs(date_1 - date_2)
        
        if date_diff > max_date_dif:
            rows_to_remove.append(i)

    #Removing The Pivot Rows That Don't Meet The Requirements.
    result = selected.drop(rows_to_remove)

    return result

### **_Plotting_**

In [26]:
def plot_sr(data, pol, por, level_dif, max_length):

    #Calculating Pivot Highs & Lows.
    PHs, PLs = get_pivot_points(data, pol, por)

    #Calculating Resistance Pivots.
    selected_high_points = get_sr(PHs, "Pivot High Prices", "Pivot High Dates", level_dif, max_length)
    selected_high_points["Average Price"] = (selected_high_points["Pivot High Prices_1"] + selected_high_points["Pivot High Prices_2"]) / 2

    #Calculating Support Pivots.
    selected_low_points = get_sr(PLs, "Pivot Low Prices", "Pivot Low Dates", level_dif, max_length)
    selected_low_points["Average Price"] = (selected_low_points["Pivot Low Prices_1"] + selected_low_points["Pivot Low Prices_2"]) / 2

    #Creating The Figure
    fig = go.Figure()

    #Candlestick Data.
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data["Open"],
                                 high = data["High"],
                                 low = data["Low"],
                                 close = data["Close"],
                                 name = "Candlesticks"))
    #Regular Pivot Highs.
    fig.add_trace(go.Scatter(x = PHs["Pivot High Dates"],
                             y = PHs["Pivot High Prices"],
                             mode = 'markers',
                             marker = dict(color = 'purple', size = 8),
                             name = 'Pivot Highs'))

    #Regular Pivot Lows.
    fig.add_trace(go.Scatter(x = PLs["Pivot Low Dates"],
                             y = PLs["Pivot Low Prices"],
                             mode = 'markers',
                             marker = dict(color = 'blue', size = 8),
                             name = 'Pivot Lows'))

    #Resistance Pivots.
    fig.add_trace(go.Scatter(x=selected_high_points["Pivot High Dates_1"],
                             y=selected_high_points["Pivot High Prices_1"],
                             mode='markers',
                             marker=dict(color='red', size=8),
                             name='Selected Pivot Highs'))

    #Resistance Pivots.
    fig.add_trace(go.Scatter(x=selected_high_points["Pivot High Dates_2"],
                             y=selected_high_points["Pivot High Prices_2"],
                             mode='markers',
                             marker=dict(color='red', size=8),
                             showlegend=False))

    #Support Pivots.
    fig.add_trace(go.Scatter(x=selected_low_points["Pivot Low Dates_1"],
                             y=selected_low_points["Pivot Low Prices_1"],
                             mode='markers',
                             marker=dict(color='green', size=8),
                             name='Selected Pivot Lows'))

    #Support Pivots.
    fig.add_trace(go.Scatter(x=selected_low_points["Pivot Low Dates_2"],
                             y=selected_low_points["Pivot Low Prices_2"],
                             mode='markers',
                             marker=dict(color='green', size=8),
                             showlegend=False))

    #Resistance Lines.
    for i in range(len(selected_high_points)):
        max_date = max(pltr.index)
        fig.add_shape(
            type="line",
            x0=selected_high_points["Pivot High Dates_1"].iloc[i],
            y0=selected_high_points["Average Price"].iloc[i],
            x1=selected_high_points["Pivot High Dates_2"].iloc[i],
            y1=selected_high_points["Average Price"].iloc[i],
            line=dict(color="rgba(255, 0, 0, 0.3)", dash = "solid"),
            name="Potential Resistance",
            xref="x",
            yref="y"
        )

    #Support Lines.
    for i in range(len(selected_low_points)):
        max_date = max(pltr.index)
        fig.add_shape(
            type="line",
            x0=selected_low_points["Pivot Low Dates_1"].iloc[i],
            y0=selected_low_points["Average Price"].iloc[i],
            x1=selected_low_points["Pivot Low Dates_2"].iloc[i],
            y1=selected_low_points["Average Price"].iloc[i],
            line=dict(color="rgba(0, 128, 0, 0.3)", dash = "solid"),
            name="Potential Support",
            xref="x",
            yref="y"
        )

    #Layout Settings.
    fig.update_layout(title='Potential Support & Resistance Lines',
                      height=600,
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    #Displaying The Final Figure.
    fig.show()

In [27]:
plot_sr(pltr, 10, 10, 0.5, 150)

#### **_Portable Plotting Function_**

**_Contains All Functions In 1_**

In [153]:
def plot_sr(data, pol, por, level_dif, max_length, recent_test_window, recent_test_threshold, title):

    #Pivot Functions
    def get_pivot_points(data, preferred_order_left = 1, preferred_order_right = 1):
    
        def identify_pivot_points(data, preferred_order_left=1, preferred_order_right=1):
            if preferred_order_left < 1 or preferred_order_right < 1:
                raise ValueError("Preferred pivot order should be 1 or greater.")

            if "High" not in data.columns or "Low" not in data.columns:
                raise ValueError("Input DataFrame must contain 'High' and 'Low' columns.")

            pivot_column_name = f"Pivot ({preferred_order_left}, {preferred_order_right}) Order"
            data[pivot_column_name] = np.nan

            def is_pivot(high_val, low_val, order_left, order_right):
                left_highs = data["High"].iloc[i - order_left:i]
                right_highs = data["High"].iloc[i + 1:i + order_right + 1]
                left_lows = data["Low"].iloc[i - order_left:i]
                right_lows = data["Low"].iloc[i + 1:i + order_right + 1]

                if high_val > left_highs.max() and high_val > right_highs.max():
                    return "High"
                elif low_val < left_lows.min() and low_val < right_lows.min():
                    return "Low"
                else:
                    return np.nan

            for i in range(preferred_order_left, len(data) - preferred_order_right):
                high_val = data["High"].iloc[i]
                low_val = data["Low"].iloc[i]
                pivot_type = is_pivot(high_val, low_val, preferred_order_left, preferred_order_right)
                if pivot_type:
                    data.iloc[i, data.columns.get_loc(pivot_column_name)] = pivot_type

            data = data.dropna(how='all')

            return data
    
        def get_pivot_high_dates(data, preferred_order=1):
            pivot_column_name = f"Pivot {preferred_order} Order"
    
            # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
            pivot_high_df = data.loc[data[pivot_column_name] == "High"]
    
            # Extract the dates from the index and store them in a list
            pivot_high_dates = pivot_high_df.index.tolist()
    
            return pivot_high_dates
        def get_pivot_low_dates(data, preferred_order=1):
            pivot_column_name = f"Pivot {preferred_order} Order"
    
            # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
            pivot_low_df = data.loc[data[pivot_column_name] == "Low"]
    
            # Extract the dates from the index and store them in a list
            pivot_low_dates = pivot_low_df.index.tolist()
    
            return pivot_low_dates
        def get_pivot_high_prices(data, preferred_order=1):
            pivot_column_name = f"Pivot {preferred_order} Order"
    
            # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
            pivot_high_df = data.loc[data[pivot_column_name] == "High"]
    
            # Extract the "High" values and store them in a list
            pivot_high_prices = pivot_high_df["High"].tolist()
    
            return pivot_high_prices
        def get_pivot_low_prices(data, preferred_order=1):
            pivot_column_name = f"Pivot {preferred_order} Order"
    
            # Filter rows with "High" values in the "Pivot {preferred_order} Order" column
            pivot_low_df = data.loc[data[pivot_column_name] == "Low"]
    
            # Extract the "High" values and store them in a list
            pivot_low_prices = pivot_low_df["Low"].tolist()
    
            return pivot_low_prices

        pivots = identify_pivot_points(data, preferred_order_left, preferred_order_right)

        order_tuple = tuple([preferred_order_left, preferred_order_right])

        P_h_dates = get_pivot_high_dates(pivots, order_tuple)
        P_h_prices = get_pivot_high_prices(pivots, order_tuple)
        P_l_dates = get_pivot_low_dates(pivots, order_tuple)
        P_l_prices = get_pivot_low_prices(pivots, order_tuple)

        return pd.DataFrame(dict({"Pivot High Dates":P_h_dates, "Pivot High Prices":P_h_prices})), pd.DataFrame({"Pivot Low Dates":P_l_dates, "Pivot Low Prices":P_l_prices})

    def get_sr(data, price_column, date_column, min_price_dif, max_date_dif_days):

        #Empty List To Be Filled With Pairs That Meet The Required Difference.
        pairs = []

        #Converting The Price & Date Columns To Workable Lists.
        prices = data[price_column].tolist()
        dates = data[date_column].tolist()

        #Looping Through A Loop To Compare All Price Differences.
        for i in range(len(prices)):
            for j in range(i + 1, len(prices)):
                if abs(prices[i] - prices[j]) < min_price_dif:
                    pairs.append((dates[i], prices[i], dates[j], prices[j]))

        #Creating A DataFrame With Pairs Of Selected Pivot Prices And Their Dates.
        selected = pd.DataFrame(pairs, columns=[date_column + "_1", price_column + "_1", date_column + "_2", price_column + "_2"])

        #Convert max_date_dif_days To Timedelta
        max_date_dif = pd.Timedelta(days=max_date_dif_days)

        #Empty List To Be Filled With Rows That Don't Meet A Second Set Of Requirements
        rows_to_remove = []

        #Looping Through The Dates Of The Selected Pivots And Comparing Their Absolute Difference With A Parameter.
        for i in range(len(selected)):
            date_1 = pd.to_datetime(selected[date_column + "_1"].iloc[i])
            date_2 = pd.to_datetime(selected[date_column + "_2"].iloc[i])
        
            date_diff = abs(date_1 - date_2)
        
            if date_diff > max_date_dif:
                rows_to_remove.append(i)

        #Removing The Pivot Rows That Don't Meet The Requirements.
        result = selected.drop(rows_to_remove)

        return result
    
    def recent_tests(data, PHs, PLs, window, threshold):
        last_window_lows = data["Low"].iloc[-window:]
        last_window_highs = data["High"].iloc[-window:]

        abs_low_diff = abs(last_window_lows - PLs["Pivot Low Prices"].iloc[-1])
        abs_high_diff = abs(last_window_highs - PHs["Pivot High Prices"].iloc[-1])

        low_data = pd.DataFrame(dict({"Price":last_window_lows, "Abs Diff":abs_low_diff}), index = last_window_lows.index)
        high_data = pd.DataFrame(dict({"Price":last_window_highs, "Abs Diff":abs_high_diff}), index = last_window_highs.index)

        rows_to_remove_lows = []
        rows_to_remove_highs = []

        for i in range(len(low_data)):
            if low_data["Abs Diff"].iloc[i] > threshold:
                rows_to_remove_lows.append(low_data.index[i])

        low_tests = low_data.drop(rows_to_remove_lows)

        for i in range(len(high_data)):
            if high_data["Abs Diff"].iloc[i] > threshold:
                rows_to_remove_highs.append(high_data.index[i])

        high_tests = high_data.drop(rows_to_remove_highs)

        cleanest_low_test = None
        if len(low_tests) > 0:
            cleanest_low_test_idx = low_tests["Abs Diff"].idxmin()
            cleanest_low_test = low_tests.loc[cleanest_low_test_idx]

        cleanest_high_test = None
        if len(high_tests) > 0:
            cleanest_high_test_idx = high_tests["Abs Diff"].idxmin()
            cleanest_high_test = high_tests.loc[cleanest_high_test_idx]

        return low_tests, cleanest_low_test, high_tests, cleanest_high_test

    #Calculating Pivot Highs & Lows.
    PHs, PLs = get_pivot_points(data, pol, por)

    #Calculating Resistance Pivots.
    selected_high_points = get_sr(PHs, "Pivot High Prices", "Pivot High Dates", level_dif, max_length)
    selected_high_points["Average Price"] = (selected_high_points["Pivot High Prices_1"] + selected_high_points["Pivot High Prices_2"]) / 2

    #Calculating Support Pivots.
    selected_low_points = get_sr(PLs, "Pivot Low Prices", "Pivot Low Dates", level_dif, max_length)
    selected_low_points["Average Price"] = (selected_low_points["Pivot Low Prices_1"] + selected_low_points["Pivot Low Prices_2"]) / 2

    #Calculating Recent Tests.
    low_tests, cleanest_low_test, high_tests, cleanest_high_test = recent_tests(data, PHs, PLs, recent_test_window, recent_test_threshold)
    
    #Creating The Figure.
    fig = go.Figure()

    #Candlestick Data.
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data["Open"],
                                 high = data["High"],
                                 low = data["Low"],
                                 close = data["Close"],
                                 name = "Candlesticks"))
    #Regular Pivot Highs.
    fig.add_trace(go.Scatter(x = PHs["Pivot High Dates"],
                             y = PHs["Pivot High Prices"],
                             mode = 'markers',
                             marker = dict(color = 'purple', size = 8),
                             name = 'Pivot Highs'))

    #Regular Pivot Lows.
    fig.add_trace(go.Scatter(x = PLs["Pivot Low Dates"],
                             y = PLs["Pivot Low Prices"],
                             mode = 'markers',
                             marker = dict(color = 'blue', size = 8),
                             name = 'Pivot Lows'))

    #Resistance Pivots.
    fig.add_trace(go.Scatter(x=selected_high_points["Pivot High Dates_1"],
                             y=selected_high_points["Pivot High Prices_1"],
                             mode='markers',
                             marker=dict(color='red', size=8),
                             name='Selected Pivot Highs'))

    #Resistance Pivots.
    fig.add_trace(go.Scatter(x=selected_high_points["Pivot High Dates_2"],
                             y=selected_high_points["Pivot High Prices_2"],
                             mode='markers',
                             marker=dict(color='red', size=8),
                             showlegend=False))

    #Support Pivots.
    fig.add_trace(go.Scatter(x=selected_low_points["Pivot Low Dates_1"],
                             y=selected_low_points["Pivot Low Prices_1"],
                             mode='markers',
                             marker=dict(color='green', size=8),
                             name='Selected Pivot Lows'))

    #Support Pivots.
    fig.add_trace(go.Scatter(x=selected_low_points["Pivot Low Dates_2"],
                             y=selected_low_points["Pivot Low Prices_2"],
                             mode='markers',
                             marker=dict(color='green', size=8),
                             showlegend=False))

    #Resistance Lines.
    for i in range(len(selected_high_points)):
        max_date = max(pltr.index)
        fig.add_shape(
            type="line",
            x0=selected_high_points["Pivot High Dates_1"].iloc[i],
            y0=selected_high_points["Average Price"].iloc[i],
            x1=selected_high_points["Pivot High Dates_2"].iloc[i],
            y1=selected_high_points["Average Price"].iloc[i],
            line=dict(color="rgba(255, 0, 0, 0.3)", dash = "solid"),
            name="Potential Resistance",
            xref="x",
            yref="y"
        )

    #Support Lines.
    for i in range(len(selected_low_points)):
        max_date = max(pltr.index)
        fig.add_shape(
            type="line",
            x0=selected_low_points["Pivot Low Dates_1"].iloc[i],
            y0=selected_low_points["Average Price"].iloc[i],
            x1=selected_low_points["Pivot Low Dates_2"].iloc[i],
            y1=selected_low_points["Average Price"].iloc[i],
            line=dict(color="rgba(0, 128, 0, 0.3)", dash = "solid"),
            name="Potential Support",
            xref="x",
            yref="y"
        )

    #Recent Tests.
    fig.add_trace(go.Scatter(mode = "markers",
                             x = low_tests.index,
                             y = low_tests["Price"],
                             marker = dict(color = "orange", size = 8),
                             name = "Recent Support Tests"))
    
    fig.add_trace(go.Scatter(mode = "markers",
                             x = high_tests.index,
                             y = high_tests["Price"],
                             marker = dict(color = "orange", size = 8),
                             name = "Recent Resistance Tests"))
    
    #Plotting Recent Tests As Support And Resistance.
    if cleanest_low_test is not None:
        
        potential_support_average = (PLs["Pivot Low Prices"].iloc[-1] + cleanest_low_test[0]) / 2
        fig.add_shape(
            type="line",
            x0=PLs["Pivot Low Dates"].iloc[-1],
            y0=potential_support_average,
            x1=cleanest_low_test.name,
            y1=potential_support_average,
            line=dict(color="rgba(0, 128, 128, 0.3)", dash = "solid"),
            name="Potential Support",
            xref="x",
            yref="y"
            )
    
    if cleanest_high_test is not None:
        
        potential_resistance_average = (PHs["Pivot High Prices"].iloc[-1] + cleanest_high_test[0]) / 2
        fig.add_shape(
            type="line",
            x0=PHs["Pivot High Dates"].iloc[-1],
            y0=potential_resistance_average,
            x1=cleanest_high_test.name,
            y1=potential_resistance_average,
            line=dict(color="rgba(0, 128, 128, 0.3)", dash = "solid"),
            name="Potential Resistance",
            xref="x",
            yref="y"
            )
    
    #Layout Settings.
    fig.update_layout(title=title,
                      height=600,
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    #Displaying The Final Figure.
    fig.show()

In [174]:
plot_sr(data=darr, pol=7, por=7, level_dif=1, max_length=356, recent_test_window=10, recent_test_threshold=1, title="DAR")

KeyboardInterrupt: 

### **_Add Code So That When Price Gets Near The Most Recent Pivot Low, A Yellow Dot Is Plotted And Vice Versa For Pivot Highs_**